In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.datasets import load_iris
#train test split
from sklearn.cross_validation import train_test_split
#implementing decision tree
from sklearn.tree import DecisionTreeClassifier

# cross validation
from sklearn.cross_validation import cross_val_score

# logistic regression
from sklearn.linear_model import LogisticRegression

# random forest

from sklearn.ensemble import RandomForestClassifier

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
M= pd.read_csv('MNIST.csv')

In [3]:

M.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
x = M.drop('label', axis=1)
y=M["label"]

In [5]:
xtrain, xtest, ytrain, ytest= train_test_split(x,y, test_size=0.30, random_state=123)

In [6]:
tree= DecisionTreeClassifier()

In [7]:
mtree= tree.fit(xtrain, ytrain)

In [8]:
tree_predict= tree.predict(xtest)

In [9]:
print(metrics.accuracy_score(ytest, tree_predict)) # bqse line prediction

0.8554761904761905


In [22]:
from sklearn.ensemble import AdaBoostClassifier
boost= AdaBoostClassifier(base_estimator=tree, n_estimators=5, learning_rate=1)

In [12]:
boost_model= boost.fit(xtrain, ytrain)
boost_predict= boost_model.predict(xtest)
print(metrics.accuracy_score(ytest,boost_predict)) # this means we got to tune our decision tree so that accuracy increases

0.8511904761904762


In [13]:
rf= RandomForestClassifier()
mrf= rf.fit(xtrain, ytrain)
rf_predict= rf.predict(xtest)
print(metrics.accuracy_score(ytest, rf_predict))

0.9353968253968254


In [17]:
#averaging scores
tree_prob= tree.predict_proba(xtest)
rf_prob= rf.predict_proba(xtest)
scores_prob=(tree_prob+rf_prob)/2
print(scores_prob)

[[0.   0.   0.9  ... 0.   0.   0.  ]
 [0.   0.   0.95 ... 0.   0.   0.05]
 [0.95 0.   0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   1.  ]
 [0.05 0.   0.55 ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.05]]


In [15]:
scores_rf= cross_val_score(rf, x, y, scoring= 'accuracy', cv=5)
print(scores_rf)

[0.93503867 0.93930739 0.93499226 0.93878766 0.94056694]


In [18]:
scores_boost= cross_val_score(boost, x, y, scoring= 'accuracy', cv=5)
print(scores_boost)

[0.85556217 0.85279067 0.85462555 0.84958914 0.85100048]


In [19]:
scores_tree= cross_val_score(tree, x, y, scoring= 'accuracy', cv=5)


In [21]:
print(scores_tree)

[0.8579417  0.85374271 0.85462555 0.85185185 0.85385898]


here ada boost can be combined with rf to get better results

In [23]:
boost_rf= AdaBoostClassifier(base_estimator=rf, n_estimators=5, learning_rate=1)
boost_rf_model= boost_rf.fit(xtrain, ytrain)
boost_rf_predict= boost_rf_model.predict(xtest)
print(metrics.accuracy_score(ytest,boost_rf_predict)) 

0.9498412698412698


In [27]:
tree_parameters={
    
    'criterion': ['gini', 'entropy'],
    'max_depth': [2,3,4,5,6],
    'max_features': ['auto', 'sqrt', 'log2'] ,
    'splitter': ['best', 'random'],
    'min_samples_split': [10,20,30,50],
    'random_state':[123]
    
}

In [28]:
from sklearn.model_selection import GridSearchCV

In [45]:
grid_search= GridSearchCV(tree, param_grid=tree_parameters, cv=10, scoring='accuracy')

In [46]:
search_model= grid_search.fit(xtrain, ytrain)

In [47]:
grid_search.best_params_

{'criterion': 'gini',
 'max_depth': 6,
 'max_features': 'auto',
 'min_samples_split': 10,
 'random_state': 123,
 'splitter': 'best'}

In [48]:
newtree= DecisionTreeClassifier(criterion='gini',max_depth= 6,max_features='auto',min_samples_split=10,random_state=1,splitter= 'best')
new_tree=newtree.fit(xtrain, ytrain)
newtree_pred= newtree.predict(xtest)

In [49]:
print(metrics.accuracy_score(ytest, newtree_pred)) 

0.609920634920635


In [50]:
boost1= AdaBoostClassifier(base_estimator=newtree, n_estimators=5, learning_rate=1)
boost1= boost1.fit(xtrain, ytrain)
boost1_predict= boost_rf_model.predict(xtest)
print(metrics.accuracy_score(ytest,boost1_predict)) 

0.9498412698412698


In [52]:
# apply ensemble voting classifier
from sklearn.ensemble import VotingClassifier
voting_model= VotingClassifier([('tree', mtree), ("Rf", mrf)], voting= 'hard')
voting_model.fit(xtrain, ytrain)
voting_predict= voting_model.predict(xtest)
print(metrics.accuracy_score(ytest, voting_predict)) 

0.8936507936507937


C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
